In [6]:
import pandas as pd

# 최종 스키마(두 컬럼 포함)
KEEP = ["id","title","detail","condition","is_completed","price","location","source","model","model_type"]

# (파일 경로, source 라벨)
files = [
    ("../../data/test/daangn_clean.csv", "daangn"),
    ("../../data/test/bungaejangter_clean.csv", "bungae"),
    ("../../data/test/naver_clean.csv", "naver"),
]

def prep(path: str, source: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df.columns = df.columns.str.lower().str.strip()
    # 동일 컬럼명만 사용 (model, model_type도 있으면 포함)
    cols = ["id"] + [c for c in KEEP if c not in ("id","source") and c in df.columns]
    df = (
        df[cols]
        .dropna(subset=["id"])
        .astype({"id": "string"})
        .drop_duplicates(subset=["id"], keep="last")
        .assign(source=source)
        .set_index("id")
    )
    return df

dfs = [prep(p, s) for p, s in files]

# 베이스(뒤 파일이 앞 파일 값을 '값이 있는 경우'에만 덮어씀)
base = pd.DataFrame(index=pd.Index([], name="id"),
                    columns=[c for c in KEEP if c != "id"])

for d in dfs:
    base = base.reindex(base.index.union(d.index))
    base.update(d)  # d의 비-결측 값만 반영

out = base.reset_index()

# model / model_type: 있으면 유지, 없거나 NaN만 빈문자 채움
for c in ["model", "model_type"]:
    if c not in out.columns:
        out[c] = ""
    else:
        out[c] = out[c].fillna("")

# 컬럼 순서 정리 + 저장(인덱스 제외)
out = out[KEEP]
out.to_csv("data_0825.csv", index=False, encoding="utf-8-sig")
print("saved merged.csv:", len(out), "rows")

saved merged.csv: 5127 rows
